In [1]:
import json
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
import shap
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from dotenv import load_dotenv
import os
import gc
import time
warnings.filterwarnings('ignore')

#debugging용 설정
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
torch.autograd.set_detect_anomaly(True)

In [2]:
# 1. 모델 및 토크나이저 로드
print("모델 및 토크나이저 로딩 중...")
model_name = "google/gemma-3-27b-it"  # Gemma 3 27B 모델 (instruction tuned)

# .env 파일에서 환경 변수 로드
load_dotenv()
token = os.getenv("HUGGINGFACE_TOKEN")

# CUDA 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"CUDA 메모리 정리 완료. 사용 가능한 GPU: {torch.cuda.device_count()}개")
    print(f"현재 GPU 메모리 사용량: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token
)
tokenizer.pad_token = tokenizer.eos_token  # 패딩 토큰 설정

# 모델 로드 시도
try:
    print(f"27B 모델 로드 중...")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",  # 자동으로 적절한 장치에 할당
        torch_dtype=torch.float32,
        token=token
    )
    device = next(model.parameters()).device
    print(f"모델 로드 완료 (device: {device})")
except Exception as e:
    print(f"GPU 로드 실패: {e}")
    print("CPU로 대체 모델 로드 중...")
    
    # 메모리 정리
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
    
    # CPU로 다시 시도
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="cpu",
        torch_dtype=torch.float32,
        token=token
    )
    device = torch.device("cpu")
    print(f"CPU 모드로 모델 로드 완료 (device: {device})")

# sentence embedding 모델 로드 (다국어 지원 모델)
print("임베딩 모델 로딩 중...")
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
print("임베딩 모델 로드 완료")

모델 및 토크나이저 로딩 중...
CUDA 메모리 정리 완료. 사용 가능한 GPU: 1개
현재 GPU 메모리 사용량: 0.00 MB
27B 모델 로드 중...


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


모델 로드 완료 (device: cuda:0)
임베딩 모델 로딩 중...
임베딩 모델 로드 완료


In [3]:
# 2. 프롬프트 기반 text generation 함수
def summarize_text(text):
    """텍스트를 요약하는 함수"""
    if not text or len(text.strip()) == 0:
        return ""
    
    # Gemma 모델용 instruction 형식 프롬프트
    prompt = f"<start_of_turn>user\n경제금융 뉴스 기사를 요약해주세요. 중요한 단어 및 내용인 무엇인지 고려해서 요약해주세요.\n\n기사: {text}<end_of_turn>\n<start_of_turn>model\n"
    
    # 토크나이징 및 입력 준비
    max_input_length = 2048-128
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_input_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    print("[DEBUG] input_ids shape:", inputs["input_ids"].shape)
    print("[DEBUG] input_ids (앞 30개):", inputs["input_ids"][0][:30]) 
    
    # 생성 설정
    gen_config = {
        "max_new_tokens": 64,
        "do_sample": False,
        "temperature": 0.7,
        "top_p": 0.95,
        "repetition_penalty": 1.1,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.pad_token_id,
        "num_return_sequences": 1
    }
    
    # 요약 생성
    with torch.no_grad():
        try:
            print(">>> input shape:", inputs["input_ids"].shape)

            output = model.generate(**inputs, **gen_config)  # GPU에서 생성
            if output.dim() > 1:
                output = output[0]  # 첫 번째 결과만 사용 (batch_size=1)

            if torch.isnan(output).any() or torch.isinf(output).any():
                print("[!] generate() 결과에 nan 또는 inf 포함됨 — 디코딩 중단")
                return ""

            output = output.to("cpu")  # CPU로 옮겨서 안전하게 디코딩
            full_response = tokenizer.decode(output, skip_special_tokens=False)  # ✅ 여기 수정!

        except Exception as e:
            print(f"[!] generate() 오류: {e}")
            return ""
        
    # 응답 파싱 (모델 응답 부분만 추출)
    try:
        # Gemma 응답 형식에서 모델 응답 부분만 추출
        if "<start_of_turn>model" in full_response:
            response_parts = full_response.split("<start_of_turn>model\n")[1]
            if "<end_of_turn>" in response_parts:
                summary = response_parts.split("<end_of_turn>")[0].strip()
            else:
                summary = response_parts.strip()
        else:
            # 프롬프트 제거
            summary = full_response.replace(prompt, "").strip()
            
        return summary
    except Exception as e:
        print(f"요약 파싱 오류: {e}")
        # 오류 발생 시 전체 응답에서 프롬프트 부분 제거 시도
        return full_response.replace(prompt, "").strip()


In [4]:
# 3. 데이터셋 가져오기
def load_dataset(file_path, max_samples=50):
    """JSON 파일에서 뉴스 문서와 요약 데이터셋 로드"""
    print(f"데이터셋 로딩 중: {file_path}")
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 데이터셋 구조 확인
        if "documents" in data:
            # 문서 데이터 추출
            documents = data["documents"]
            processed_data = []
            
            for doc in documents:
                # 원문 텍스트 추출 (text는 배열 구조)
                original_text = ""
                if "text" in doc:
                    # 문장들을 하나의 텍스트로 합치기
                    for paragraph in doc["text"]:
                        for sent_obj in paragraph:
                            if "sentence" in sent_obj:
                                original_text += sent_obj["sentence"] + " "
                
                # 요약문 추출
                summary = ""
                if "abstractive" in doc and doc["abstractive"]:
                    summary = doc["abstractive"][0] if isinstance(doc["abstractive"], list) else doc["abstractive"]
                
                # 필요한 정보만 추출하여 데이터프레임용 딕셔너리 생성
                processed_doc = {
                    "id": doc.get("id", ""),
                    "title": doc.get("title", ""),
                    "text": original_text.strip(),
                    "summary": summary,
                    "category": doc.get("category", ""),
                    "media_name": doc.get("media_name", "")
                }
                
                processed_data.append(processed_doc)
            
            # 데이터프레임 생성
            df = pd.DataFrame(processed_data)
            
        else:
            # 다른 구조의 JSON 처리
            print("표준 구조가 아닌 JSON 파일입니다.")
            if isinstance(data, list):
                df = pd.DataFrame(data)
            else:
                df = pd.DataFrame([data])
            
            # 컬럼 이름 확인 및 변환
            if 'article' in df.columns and 'text' not in df.columns:
                df['text'] = df['article']
            if 'summary' not in df.columns and 'abstractive' in df.columns:
                df['summary'] = df['abstractive']
        
        # 빈 텍스트나 요약이 있는 행 제거
        df = df.dropna(subset=['text'])
        df = df[df['text'].str.strip() != '']
        
        # 최대 샘플 수 제한
        if len(df) > max_samples:
            df = df.sample(max_samples, random_state=42)
        
        print(f"데이터셋 로딩 완료: {len(df)} 샘플")
        return df
    
    except Exception as e:
        print(f"데이터셋 로딩 실패: {e}")
        # 예시 데이터 생성
        print("예시 데이터 사용")
        example_data = {
            'text': [
                "한국은행이 통화정책 결정 회의에서 기준금리를 동결했다. 한국은행은 지난해 4분기 이후 계속된 경기침체의 영향으로 고용 시장이 위축되고 있으며, 물가상승률은 목표 수준으로 안정되고 있다고 판단했다. 시장 전문가들은 한국 경제의 회복세가 예상보다 더디게 진행되고 있어 금리 인하 가능성도 있다고 전망했다.",
                "금융위원회는 오늘 가계부채 관리 방안을 발표했다. 주요 내용은 총부채원리금상환비율(DSR) 규제를 40%로 강화하고, 다주택자에 대한 주택담보대출 제한을 확대하는 것이다. 또한 실수요자에 대한 대출 지원은 확대하되, 투기 목적의 대출에는 제재를 강화하는 투트랙 전략을 펼치기로 했다."
            ],
            'summary': [
                "한국은행이 통화정책 결정 회의에서 기준금리 동결, 경기침체로 인한 고용시장 위축과 물가 안정 판단",
                "금융위, 가계부채 관리방안 발표 - DSR 40% 강화, 다주택자 대출제한 확대, 실수요자 지원 확대"
            ]
        }
        return pd.DataFrame(example_data)

In [11]:
# 4. SHAP용 파이프라인 및 Explainer 구성
class SummarizationPipeline:
    def __init__(self, model, tokenizer, embedder):
        self.model = model
        self.tokenizer = tokenizer
        self.embedder = embedder
        self.original_text = None
        self.reference_summary = None
        self.perturbation_count = 0
    
    def set_reference(self, text, summary=None):
        """원본 텍스트와 참조 요약 설정"""
        self.original_text = text
        self.perturbation_count = 0
        
        if summary is None:
            # 참조 요약이 없으면 모델로 생성
            print("참조 요약 생성 중...")
            self.reference_summary = summarize_text(text)
            print(f"생성된 참조 요약: {self.reference_summary}")
        else:
            self.reference_summary = summary
            print(f"제공된 참조 요약: {self.reference_summary}")
        
        # 참조 요약 임베딩 미리 계산
        self.reference_embedding = self.embedder.encode(
            self.reference_summary, convert_to_tensor=True
        )

    
    def __call__(self, texts):
        """
        SHAP용 호출 함수. Perturbation된 텍스트 목록을 받아 각각의 요약 품질 점수 반환
        """
        if not isinstance(texts, list):
            texts = [texts]
        
        batch_size = min(2, len(texts))  # 배치 크기 줄임(메모리 고려)
        all_scores = []
        
        # 진행 상황 출력
        # tqdm 진행 바 설정
        total = getattr(self, "_expected_perturbations", 100)
        pbar = tqdm(total=total, desc="🔍 SHAP perturbation 진행", unit="step", leave=True)
        pbar.n = self.perturbation_count  # 이전에 진행된 수 반영
        pbar.refresh()
         
                
        # 배치 단위로 처리
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            batch_scores = []
            
            for text in batch_texts:
                # 넘파이 배열이나 텐서 처리
                if isinstance(text, (np.ndarray, torch.Tensor)):
                    text = text.tolist()

                # 리스트 → 첫 원소 꺼냄
                if isinstance(text, list) and len(text) > 0:
                    text = text[0]

                # 문자열인지 최종 확인
                if not isinstance(text, str):
                    print(f"[!] 텍스트를 문자열로 변환할 수 없음: {type(text)}")
                    batch_scores.append(0.0)
                    continue
                
                # 빈 텍스트 체크
                if not text.strip():
                    batch_scores.append(0.0)
                    continue
                    
                try:
                    # 요약 생성
                    summary = summarize_text(text)
                    
                    if not summary.strip():
                        print("[!] 요약 결과가 공백입니다. 점수 계산 생략")
                        batch_scores.append(0.0)
                        continue
                    
                    # 생성된 요약의 임베딩 계산
                    summary_embedding = self.embedder.encode(
                        summary, convert_to_tensor=True
                    )

                    # NaN 체크 및 처리
                    if torch.isnan(summary_embedding).any() or torch.isnan(self.reference_embedding).any():
                        print(f"[!] NaN detected in embeddings. 요약 텍스트: {summary}")
                        batch_scores.append(0.0)
                        continue
                    
                    # 참조 요약과의 유사도 계산 (코사인 유사도)
                    similarity = util.cos_sim(
                        summary_embedding, self.reference_embedding
                    ).item()
                    
                    # 점수 범위 보정
                    normalized_score = max(0.0, min(1.0, float(similarity)))
                    batch_scores.append(normalized_score)

                    self.perturbation_count += 1
                    pbar.update(1)
                    
                except Exception as e:
                    print(f"[!] 점수 계산 오류: {str(e)}")
                    batch_scores.append(0.0)  # 오류 시 0점 처리
            
            all_scores.extend(batch_scores)
                    
            # 🔍 디버깅 로그 추가
            print(f"[DEBUG] texts 길이: {len(texts)}")
            print(f"[DEBUG] 현재 batch_texts 길이: {len(batch_texts)}")
            print(f"[DEBUG] batch_scores 길이: {len(batch_scores)}")
            print(f"[DEBUG] 누적 all_scores 길이: {len(all_scores)}")
    
        # 무조건 입력 개수와 출력 개수 맞춰주기
        if len(all_scores) != len(texts):
            print(f"[!] Warning: 입력 수({len(texts)})와 출력 수({len(all_scores)})가 다릅니다. 0.0으로 채움")
            if len(all_scores) > len(texts):
                # 너무 많이 만든 경우 자르기
                all_scores = all_scores[:len(texts)]
            else:
                # 부족한 경우 0.0으로 채우기
                while len(all_scores) < len(texts):
                    all_scores.append(0.0)

        if len(texts) == 1:
            return [float(all_scores[0])]  # SHAP이 1개만 요청한 경우
        else:
            return [float(s) for s in all_scores]
    
    def explain_specific_tokens(self, tokens, top_n=5):
        """
        특정 토큰들의 중요도를 개별적으로 분석
        """
        results = []
        original_summary = summarize_text(self.original_text)
        
        for token in tokens:
            # 해당 토큰을 제거한 텍스트 생성
            removed_text = self.original_text.replace(token, "")
            
            # 제거 후 요약 생성
            removed_summary = summarize_text(removed_text)
            
            # 원본 요약과 제거 후 요약의 유사도 계산
            original_emb = self.embedder.encode(original_summary, convert_to_tensor=True)
            removed_emb = self.embedder.encode(removed_summary, convert_to_tensor=True)
            
            # 유사도 차이가 클수록 해당 토큰이 중요함
            similarity = util.cos_sim(original_emb, removed_emb).item()
            importance = 1.0 - similarity
            
            results.append({
                "token": token,
                "importance": importance,
                "original_summary": original_summary,
                "removed_summary": removed_summary
            })
            
        # 중요도 순으로 정렬
        results.sort(key=lambda x: x["importance"], reverse=True)
        return results[:top_n]


def analyze_with_shap(text, reference_summary=None, num_samples=100, verbose=True):
    """
    단일 텍스트에 대해 SHAP 분석 수행
    
    Args:
        text: 분석할 원문 텍스트
        reference_summary: 참조 요약 (없으면 모델로 생성)
        num_samples: SHAP 샘플링 수
        verbose: 자세한 출력 여부
    
    Returns:
        shap_values: SHAP 값
        summary: 생성된 요약
        pipeline: 분석에 사용된 파이프라인 객체
    """
    if verbose:
        print(f"\n{'='*80}\n원문 분석 시작\n{'='*80}")
        print(f"원문 (일부): {text[:200]}...")
    
    # 빈 텍스트 확인
    if not isinstance(text, str) or len(text.strip()) == 0:
        raise ValueError("분석할 텍스트가 비어 있습니다.")
    
    # 파이프라인 초기화 및 참조 설정
    pipeline = SummarizationPipeline(model, tokenizer, embedder)
    pipeline.set_reference(text, reference_summary)
    
    # SHAP Explainer 생성
    try:
        # 단어 단위로 분할
        words = text.split()
        
        if verbose:
            print(f"SHAP Explainer 초기화 중...")
            print(f"텍스트 길이: {len(text)}, 단어 수: {len(words)}")
        
        mask_token = tokenizer.pad_token or tokenizer.eos_token or "…"  # fallback
        masker = shap.maskers.Text(tokenizer=tokenizer, mask_token=mask_token)          
        
        # Partition 마스커 사용 (더 안정적)
        explainer = shap.Explainer(pipeline, masker)
        
        # 샘플 수 자동 조정 (너무 많으면 오래 걸림)
        num_features = len(words)
        adjusted_samples = min(max(2 * num_features + 1, 50), num_samples)
        
        if verbose:
            print(f"SHAP 값 계산 중 (단어 수: {num_features}, 샘플 수: {adjusted_samples})...")
        
        # SHAP 값 계산 - 단일 데이터로 명시
        shap_values = explainer([text], max_evals=30)
        
        if verbose:
            print("SHAP 분석 완료")
            print(f"SHAP 값 형태: {shap_values.values.shape}")
        
        return shap_values, pipeline.reference_summary, pipeline
        
    except Exception as e:
        print(f"SHAP 분석 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        
        print("더 단순한 설정으로 SHAP 분석 재시도...")
        try:
            # 더 단순하고 안정적인 접근방식 사용
            # 텍스트 직접 처리
            tokenized_text = text.split()
            
            # 수동으로 SHAP 값 생성
            dummy_values = np.zeros((1, len(tokenized_text)))
            
            # 각 토큰의 중요도를 간단히 계산 (임베딩 유사도 기반)
            original_summary = summarize_text(text)
            
            for i, token in enumerate(tokenized_text):
                # 해당 토큰 제거
                modified_text = ' '.join([t for j, t in enumerate(tokenized_text) if j != i])
                
                try:
                    # 수정된 텍스트로 요약 생성
                    modified_summary = summarize_text(modified_text)
                    
                    # 두 요약의 임베딩 비교
                    orig_emb = pipeline.embedder.encode(original_summary, convert_to_tensor=True)
                    mod_emb = pipeline.embedder.encode(modified_summary, convert_to_tensor=True)
                    
                    # 유사도 계산 (1 - 유사도 = 중요도)
                    similarity = util.cos_sim(orig_emb, mod_emb).item()
                    importance = 1.0 - similarity
                    
                    # 중요도 저장
                    dummy_values[0, i] = importance
                except Exception as e2:
                    print(f"토큰 '{token}' 처리 중 오류: {e2}")
                    dummy_values[0, i] = 0.0
            
            # SHAP 결과와 유사한 객체 생성
            class CustomShapValues:
                def __init__(self, values, data, feature_names):
                    self.values = values
                    self.data = data
                    self.feature_names = feature_names
                    self.output_names = ["importance"]
                    self.base_values = np.zeros(1)
            
            # 결과 반환
            shap_obj = CustomShapValues(
                values=dummy_values,
                data=np.array([text]),
                feature_names=tokenized_text
            )
            
            print("대체 SHAP 분석 완료")
            return shap_obj, original_summary, pipeline
            
        except Exception as retry_error:
            print(f"대체 SHAP 분석도 실패: {retry_error}")
            traceback.print_exc()
            raise ValueError("모든 SHAP 분석 방법이 실패했습니다.")

def print_important_features(shap_values, summary, top_n=10):
    """
    SHAP 값을 기반으로 중요 feature를 출력
    """
    # SHAP 값의 절대값을 기준으로 정렬
    token_importances = []
    
    try:
        # 데이터 형식에 따라 다르게 처리
        if hasattr(shap_values, 'data') and isinstance(shap_values.data, np.ndarray):
            text_data = shap_values.data[0]
            if isinstance(text_data, np.ndarray) and text_data.size == 1:
                text_data = text_data.item()
        else:
            # 대체 방식
            text_data = shap_values.data[0] if hasattr(shap_values, 'data') else "텍스트 데이터 없음"
        
        # 토큰 분할
        tokens = text_data.split() if isinstance(text_data, str) else []
        
        # 토큰과 SHAP 값 매핑
        for i, token in enumerate(tokens):
            if i < shap_values.values.shape[1]:  # 인덱스 범위 확인
                importance = abs(shap_values.values[0][i])
                raw_value = shap_values.values[0][i]
                token_importances.append((token, importance, raw_value))
        
        # 중요도 기준 정렬
        sorted_importances = sorted(token_importances, key=lambda x: x[1], reverse=True)
        
        print(f"\n{'='*80}")
        print(f"생성된 요약: {summary}")
        print(f"{'='*80}")
        print(f"상위 {top_n}개 중요 단어 (SHAP 기준):")
        print(f"{'단어':<15} | {'중요도':>10} | {'영향':>10} | {'해석'}")
        print(f"{'-'*60}")
        
        for token, importance, raw_value in sorted_importances[:top_n]:
            effect = "긍정적 영향" if raw_value > 0 else "부정적 영향"
            print(f"{token:<15} | {importance:>10.4f} | {raw_value:>10.4f} | {effect}")
        
        print(f"{'='*80}")
    
    except Exception as e:
        print(f"중요 특성 출력 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        print("간소화된 분석 결과 출력:")
        print(f"요약: {summary}")

In [12]:
# 5. 메인 실행 코드
if __name__ == "__main__":
    dataset_path = "./data/train_original.json" # 실제 파일 경로로 변경
    df = load_dataset(dataset_path, max_samples=10)  # 테스트용 10개만
    
    # 텍스트와 참조 요약 추출
    texts = df["text"].tolist()
    references = df["summary"].tolist() if "summary" in df.columns else [None] * len(texts)
    
    print(f"\n총 {len(texts)}개 샘플 분석 시작")
    
    # 각 샘플에 대해 SHAP 분석 수행
    for i, (text, reference) in enumerate(zip(texts, references)):
        print(f"\n샘플 {i+1}/{len(texts)} 분석:")
        
        try:
            shap_values, summary, _ = analyze_with_shap(
                text, 
                reference_summary=reference,
                num_samples=5,  # 샘플링 수 조정 (높을수록 정확하지만 느림)
                verbose=True
            )
            
            # 중요 feature 출력
            print_important_features(shap_values, summary, top_n=15)
            
        except Exception as e:
            print(f"샘플 {i+1} 분석 중 오류 발생: {e}")
            print("이 샘플은 건너뜁니다.")
            continue
    
    print("\n모든 분석 완료!")

데이터셋 로딩 중: ./data/train_original.json
데이터셋 로딩 완료: 10 샘플

총 10개 샘플 분석 시작

샘플 1/10 분석:

원문 분석 시작
원문 (일부): 최명국 송하진 지사, 방중 주요 성과로 '군산~연운항' 항로 개설 협의 꼽아 장쑤성 당서기 "바닷길 통한 협력, 적극 검토" 두 지역 인적 교류 활발, 지난해 석도 항로 증편 송하진 전북도지사가 중국 장쑤성(강소성) 방문의 주요 성과로 중국 측과의 '군산~장쑤성 연운항' 항로 개설 협의를 꼽았다. 송하진 도지사는 1일 출입기자들과 만나 "군산과 장쑤성 연운...
제공된 참조 요약: 송하진 전북도지사가 중국 장쑤성을 방문해 러우 친지앤 당서기와 새만금 산단 5공구 공동투자 활용안에 대해 협의하고 군산과 장쑤성 연운항 간 신규 여객 항로 개설을 통한 협력방안을 논의했다.
SHAP Explainer 초기화 중...
텍스트 길이: 892, 단어 수: 219
SHAP 값 계산 중 (단어 수: 219, 샘플 수: 5)...


[DEBUG] input_ids shape: torch.Size([1, 38])
[DEBUG] input_ids (앞 30개): tensor([     2,    105,   2364,    107, 179886, 200086, 234416,   9554,  68564,
         23591, 239114, 219770, 236761,  86394,  29950, 237430,  24566,  54422,
        237558,  94485,  93860, 143495,  22063,  23591, 239114, 219770, 236761,
           108, 237351, 237470], device='cuda:0')
>>> input shape: torch.Size([1, 38])


🔍 SHAP perturbation 진행:   1%|          | 1/100 [11:13<18:31:52, 673.86s/step]


[DEBUG] texts 길이: 1
[DEBUG] 현재 batch_texts 길이: 1
[DEBUG] batch_scores 길이: 1
[DEBUG] 누적 all_scores 길이: 1


[DEBUG] input_ids shape: torch.Size([1, 595])
[DEBUG] input_ids (앞 30개): tensor([     2,    105,   2364,    107, 179886, 200086, 234416,   9554,  68564,
         23591, 239114, 219770, 236761,  86394,  29950, 237430,  24566,  54422,
        237558,  94485,  93860, 143495,  22063,  23591, 239114, 219770, 236761,
           108, 237351, 237470], device='cuda:0')
>>> input shape: torch.Size([1, 595])


...

[DEBUG] texts 길이: 1
[DEBUG] 현재 batch_texts 길이: 1
[DEBUG] batch_scores 길이: 1
[DEBUG] 누적 all_scores 길이: 1


[DEBUG] input_ids shape: torch.Size([1, 306])
[DEBUG] input_ids (앞 30개): tensor([     2,    105,   2364,    107, 179886, 200086, 234416,   9554,  68564,
         23591, 239114, 219770, 236761,  86394,  29950, 237430,  24566,  54422,
        237558,  94485,  93860, 143495,  22063,  23591, 239114, 219770, 236761,
           108, 237351, 237470], device='cuda:0')
>>> input shape: torch.Size([1, 306])


🔍 SHAP perturbation 진행:   3%|▎         | 3/100 [11:11<6:01:44, 223.76s/step]
Traceback (most recent call last):
  File "/tmp/ipykernel_6468/2163360236.py", line 218, in analyze_with_shap
    shap_values = explainer([text], max_evals=30)
  File "/home/elicer/venv/lib/python3.10/site-packages/shap/explainers/_partition.py", line 173, in __call__
    return super().__call__(
  File "/home/elicer/venv/lib/python3.10/site-packages/shap/explainers/_explainer.py", line 366, in __call__
    row_result = self.explain_row(
  File "/home/elicer/venv/lib/python3.10/site-packages/shap/explainers/_partition.py", line 229, in explain_row
    self.owen(fm, self._curr_base_value, f11, max_evals - 2, outputs, fixed_context, batch_size, silent)
  File "/home/elicer/venv/lib/python3.10/site-packages/shap/explainers/_partition.py", line 339, in owen
    fout = fm(batch_masks)
  File "/home/elicer/venv/lib/python3.10/site-packages/shap/utils/_masked_model.py", line 67, in __call__
    return self._full_mask

[DEBUG] texts 길이: 1
[DEBUG] 현재 batch_texts 길이: 1
[DEBUG] batch_scores 길이: 1
[DEBUG] 누적 all_scores 길이: 1
SHAP 분석 중 오류 발생: The model produced 1 output rows when given 2 input rows! Check the implementation of the model you provided for errors.
더 단순한 설정으로 SHAP 분석 재시도...
[DEBUG] input_ids shape: torch.Size([1, 595])
[DEBUG] input_ids (앞 30개): tensor([     2,    105,   2364,    107, 179886, 200086, 234416,   9554,  68564,
         23591, 239114, 219770, 236761,  86394,  29950, 237430,  24566,  54422,
        237558,  94485,  93860, 143495,  22063,  23591, 239114, 219770, 236761,
           108, 237351, 237470], device='cuda:0')
>>> input shape: torch.Size([1, 595])
[DEBUG] input_ids shape: torch.Size([1, 592])
[DEBUG] input_ids (앞 30개): tensor([     2,    105,   2364,    107, 179886, 200086, 234416,   9554,  68564,
         23591, 239114, 219770, 236761,  86394,  29950, 237430,  24566,  54422,
        237558,  94485,  93860, 143495,  22063,  23591, 239114, 219770, 236761,
           108, 23

KeyboardInterrupt: 